In [ ]:
import psycopg2
import csv
from config.db_config import config

class DataInserter:
    def __init__(self):
        self.conn = None

    def connect(self):
        try:
            params = config()
            self.conn = psycopg2.connect(**params)
        except (Exception, psycopg2.DatabaseError) as error:
            print("Error occurred during connection:", error)
    
    def execute_query(self, query, values):
        try:
            with self.conn.cursor() as cur:
                cur.execute(query, values)
        except psycopg2.Error as error:
            print("Error occurred during query execution:", error)
    
    def insert_data(self, table_name, data_path, column_names, conflict_columns, update_columns=None):
        try:
            self.connect()
            with open(data_path, 'r') as file:
                reader = csv.reader(file)
                next(reader)  # Skip the header row

                for row in reader:
                    placeholders = ', '.join(['%s'] * len(column_names))
                    conflict_placeholders = ', '.join(k for k in conflict_columns)

                    if update_columns:
                        update_placeholders = ', '.join([f"{col} = EXCLUDED.{col}" for col in update_columns])
                        query = f"INSERT INTO {table_name} ({', '.join(column_names)})\
                                  VALUES ({placeholders})\
                                  ON CONFLICT ({conflict_placeholders})\
                                  DO UPDATE SET {update_placeholders};"
                    else:
                        query = f"INSERT INTO {table_name} ({', '.join(column_names)})\
                                  VALUES ({placeholders})\
                                  ON CONFLICT ({conflict_placeholders})\
                                  DO NOTHING;"
                    
                    values = tuple(row[column_names.index(col)] for col in column_names)
                    self.execute_query(query, values)

            self.conn.commit()
            print("Data insertion completed successfully.")

        except (Exception, psycopg2.DatabaseError) as error:
            print("Error occurred during data insertion:", error)

            if self.conn is not None:
                self.conn.rollback()

        finally:
            if self.conn is not None:
                self.conn.close()
                print('Database connection closed.')
if __name__ == '__main__':
    DataInserter()

In [13]:
from tools.insert_data_tool import *

shop_profile_data_config = {
    'table_name' : 'pdata.shop_profile'
    ,'column_names' : ['name', 'score', 'comment_cnt', 'style']
    ,'conflict_columns' : ['name']
    ,'update_columns' : ['score', 'comment_cnt', 'style']
}

shop_feature_data_config = {
    'table_name' : 'pdata.shop_feature'
    ,'column_names' : ['name', 'feature']
    ,'conflict_columns' : ['name', 'feature']
    ,'update_columns' : None
}

shop_url_data_config = {
    'table_name' : 'pdata.shop_url'
    ,'column_names' : ['name', 'url']
    ,'conflict_columns' : ['name']
    ,'update_columns' : ['url']
}

shop_adr_data_config = {
    'table_name' : 'pdata.shop_adr'
    ,'column_names' : ['name', 'country', 'postalCode', 'address', 'latitude', 'longitude', 'telephone']
    ,'conflict_columns' : ['name']
    ,'update_columns' : ['country', 'postalCode', 'address', 'latitude', 'longitude', 'telephone']
}

shop_time_data_config = {
    'table_name' : 'pdata.shop_time'
    ,'column_names' : ['name', 'dayOfWeek', 'opens_time', 'closes_time']
    ,'conflict_columns' : ['name', 'dayOfWeek']
    ,'update_columns' : ['opens_time', 'closes_time']
}

shop_feature_detail_data_config = {
    'table_name' : 'pdata.shop_feature_detail'
    ,'column_names' : ['name', 'feature']
    ,'conflict_columns' : ['name', 'feature']
    ,'update_columns' : None
}

shop_product_detail_data_config = {
    'table_name' : 'pdata.shop_product_detail'
    ,'column_names' : ['name', 'product', 'price']
    ,'conflict_columns' : ['name', 'product']
    ,'update_columns' : ['price']
}



import os

main_path = os.getcwd()  # 程式所在的目錄路徑
data_path = 'data'
combine_path = os.path.join(main_path,data_path)# 檔案所在的目錄路徑

# 列出目錄下所有以指定前綴開頭的檔案
shop_profile_prefix = "shop_profile"  # 檔案名稱前綴
shop_profile_files = [f for f in os.listdir(combine_path) if f.startswith(shop_profile_prefix)]
# 將檔案按照最後修改時間排序，並取最後一個即為最新創建的shop_profile_file檔案
shop_profile_file = sorted(shop_profile_files, key=lambda x: os.path.getmtime(os.path.join(combine_path, x)), reverse=True)[0]
shop_profile_data_path = os.path.join(combine_path,shop_profile_file)
print("最新創建的shop_profile檔案是:", shop_profile_data_path)

shop_feature_prefix = "shop_feature"
shop_feature_files = [f for f in os.listdir(combine_path) if f.startswith(shop_feature_prefix)]
shop_feature_file = sorted(shop_feature_files, key=lambda x: os.path.getmtime(os.path.join(combine_path, x)), reverse=True)[0]
shop_feature_data_path = os.path.join(combine_path,shop_feature_file)
print("最新創建的shop_feature檔案是:", shop_feature_data_path)

shop_url_prefix = "shop_url"
shop_url_files = [f for f in os.listdir(combine_path) if f.startswith(shop_url_prefix)]
shop_url_file = sorted(shop_url_files, key=lambda x: os.path.getmtime(os.path.join(combine_path, x)), reverse=True)[0]
shop_url_data_path = os.path.join(combine_path,shop_url_file)
print("最新創建的shop_url檔案是:", shop_url_data_path)

shop_adr_prefix = "shop_adr"
shop_adr_files = [f for f in os.listdir(combine_path) if f.startswith(shop_adr_prefix)]
shop_adr_file = sorted(shop_adr_files, key=lambda x: os.path.getmtime(os.path.join(combine_path, x)), reverse=True)[0]
shop_adr_data_path = os.path.join(combine_path,shop_adr_file)
print("最新創建的shop_adr檔案是:", shop_adr_data_path)

shop_time_prefix = "shop_time"
shop_time_files = [f for f in os.listdir(combine_path) if f.startswith(shop_time_prefix)]
shop_time_file = sorted(shop_time_files, key=lambda x: os.path.getmtime(os.path.join(combine_path, x)), reverse=True)[0]
shop_time_data_path = os.path.join(combine_path,shop_time_file)
print("最新創建的shop_time檔案是:", shop_time_data_path)

shop_feature_detail_prefix = "shop_feature_detail"
shop_feature_detail_files = [f for f in os.listdir(combine_path) if f.startswith(shop_feature_detail_prefix)]
shop_feature_detail_file = sorted(shop_feature_detail_files, key=lambda x: os.path.getmtime(os.path.join(combine_path, x)), reverse=True)[0]
shop_feature_detail_data_path = os.path.join(combine_path,shop_feature_detail_file)
print("最新創建的shop_feature_detail檔案是:", shop_feature_detail_data_path)

shop_product_detail_prefix = "shop_product_detail"
shop_product_detail_files = [f for f in os.listdir(combine_path) if f.startswith(shop_product_detail_prefix)]
shop_product_detail_file = sorted(shop_product_detail_files, key=lambda x: os.path.getmtime(os.path.join(combine_path, x)), reverse=True)[0]
shop_product_detail_data_path = os.path.join(combine_path,shop_product_detail_file)
print("最新創建的shop_product_detail檔案是:", shop_product_detail_data_path)



data_inserter = DataInserter()
data_inserter.insert_data(shop_profile_data_config['table_name']
                          ,shop_profile_data_path
                          ,shop_profile_data_config['column_names']
                          ,shop_profile_data_config['conflict_columns']
                          ,shop_profile_data_config['update_columns'])

data_inserter.insert_data(shop_feature_data_config['table_name']
                          ,shop_feature_data_path
                          ,shop_feature_data_config['column_names']
                          ,shop_feature_data_config['conflict_columns']
                          ,shop_feature_data_config['update_columns'])

data_inserter.insert_data(shop_url_data_config['table_name']
                          ,shop_url_data_path
                          ,shop_url_data_config['column_names']
                          ,shop_url_data_config['conflict_columns']
                          ,shop_url_data_config['update_columns'])

data_inserter.insert_data(shop_adr_data_config['table_name']
                          ,shop_adr_data_path
                          ,shop_adr_data_config['column_names']
                          ,shop_adr_data_config['conflict_columns']
                          ,shop_adr_data_config['update_columns'])

data_inserter.insert_data(shop_time_data_config['table_name']
                          ,shop_time_data_path
                          ,shop_time_data_config['column_names']
                          ,shop_time_data_config['conflict_columns']
                          ,shop_time_data_config['update_columns'])


data_inserter.insert_data(shop_feature_detail_data_config['table_name']
                          ,shop_feature_detail_data_path
                          ,shop_feature_detail_data_config['column_names']
                          ,shop_feature_detail_data_config['conflict_columns']
                          ,shop_feature_detail_data_config['update_columns'])

data_inserter.insert_data(shop_product_detail_data_config['table_name']
                          ,shop_product_detail_data_path
                          ,shop_product_detail_data_config['column_names']
                          ,shop_product_detail_data_config['conflict_columns']
                          ,shop_product_detail_data_config['update_columns'])

最新創建的shop_profile檔案是: /mnt/d/Cupoy_marathon/Python 網路爬蟲實戰馬拉松/foodpanda_project/data/shop_profile_20230522222638.csv
最新創建的shop_feature檔案是: /mnt/d/Cupoy_marathon/Python 網路爬蟲實戰馬拉松/foodpanda_project/data/shop_feature_20230522222638.csv
最新創建的shop_url檔案是: /mnt/d/Cupoy_marathon/Python 網路爬蟲實戰馬拉松/foodpanda_project/data/shop_url_20230522222638.csv
最新創建的shop_adr檔案是: /mnt/d/Cupoy_marathon/Python 網路爬蟲實戰馬拉松/foodpanda_project/data/shop_adr_20230517215338.csv
最新創建的shop_time檔案是: /mnt/d/Cupoy_marathon/Python 網路爬蟲實戰馬拉松/foodpanda_project/data/shop_time_20230517215338.csv
最新創建的shop_feature_detail檔案是: /mnt/d/Cupoy_marathon/Python 網路爬蟲實戰馬拉松/foodpanda_project/data/shop_feature_detail_20230517215338.csv
最新創建的shop_product_detail檔案是: /mnt/d/Cupoy_marathon/Python 網路爬蟲實戰馬拉松/foodpanda_project/data/shop_product_detail_20230517215338.csv
Data insertion completed successfully.
Database connection closed.
Data insertion completed successfully.
Database connection closed.
Data insertion completed successfully.
Database 